# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/9510/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

['/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-01.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-02.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-03.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-04.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-05.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-06.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-08.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-09.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-10.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-11.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-12.csv',
 '/home/runner/work/dask-exa

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,962,Tim,0.3708561650659399,-0.2018945380648598
2000-01-01 00:00:01,953,Dan,-0.014814198283820135,0.1017894312023071
2000-01-01 00:00:02,975,Sarah,0.5562883104302032,-0.12588301590697593
2000-01-01 00:00:03,1023,Alice,-0.6584699378220464,0.37617193031876806
2000-01-01 00:00:04,1000,Laura,-0.7357683507143375,-0.8087217586105662
2000-01-01 00:00:05,1028,Zelda,-0.2004395950347495,0.2398104107607688
2000-01-01 00:00:06,959,Charlie,-0.6360237164139324,-0.8961537958344952
2000-01-01 00:00:07,963,Jerry,0.5035040011432743,-0.06962423049974231
2000-01-01 00:00:08,1070,Tim,-0.4312988031685163,-0.618695390486049


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1021,Patricia,-0.8900835142383949,-0.8196817327590775
2000-01-30 00:00:01,981,Ingrid,0.7001056530851171,-0.47991678572012053
2000-01-30 00:00:02,1047,Kevin,-0.8933265455973121,0.46529995885578
2000-01-30 00:00:03,1000,Ursula,-0.14619164971539056,0.5506475206454808
2000-01-30 00:00:04,1058,Xavier,-0.4297264560454781,-0.5705441995936447
2000-01-30 00:00:05,987,Charlie,0.5277905712397999,0.20218500271395268
2000-01-30 00:00:06,991,Hannah,0.31408960673525765,-0.19599174486970905
2000-01-30 00:00:07,1009,Oliver,0.4453832986988666,-0.7381898741357975
2000-01-30 00:00:08,990,Laura,-0.6119791595276691,0.40496458608864305


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,962,Tim,0.370856,-0.201895
1,2000-01-01 00:00:01,953,Dan,-0.014814,0.101789
2,2000-01-01 00:00:02,975,Sarah,0.556288,-0.125883
3,2000-01-01 00:00:03,1023,Alice,-0.658470,0.376172
4,2000-01-01 00:00:04,1000,Laura,-0.735768,-0.808722


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,962,Tim,0.370856,-0.201895
1,2000-01-01 00:00:01,953,Dan,-0.014814,0.101789
2,2000-01-01 00:00:02,975,Sarah,0.556288,-0.125883
3,2000-01-01 00:00:03,1023,Alice,-0.658470,0.376172
4,2000-01-01 00:00:04,1000,Laura,-0.735768,-0.808722


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.9 s, sys: 419 ms, total: 4.32 s
Wall time: 3.02 s


name
Alice      -0.000854
Bob         0.002710
Charlie     0.000194
Dan        -0.001115
Edith       0.000555
Frank      -0.002418
George      0.001846
Hannah     -0.001776
Ingrid     -0.002040
Jerry       0.002220
Kevin      -0.000385
Laura      -0.000259
Michael    -0.001937
Norbert     0.000206
Oliver      0.000625
Patricia   -0.000618
Quinn      -0.002781
Ray         0.002357
Sarah      -0.000837
Tim         0.001247
Ursula      0.000153
Victor      0.000058
Wendy      -0.001376
Xavier     -0.000991
Yvonne      0.002613
Zelda       0.000713
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.26 s, sys: 151 ms, total: 1.41 s
Wall time: 1.07 s


name
Alice      -0.000854
Bob         0.002710
Charlie     0.000194
Dan        -0.001115
Edith       0.000555
Frank      -0.002418
George      0.001846
Hannah     -0.001776
Ingrid     -0.002040
Jerry       0.002220
Kevin      -0.000385
Laura      -0.000259
Michael    -0.001937
Norbert     0.000206
Oliver      0.000625
Patricia   -0.000618
Quinn      -0.002781
Ray         0.002357
Sarah      -0.000837
Tim         0.001247
Ursula      0.000153
Victor      0.000058
Wendy      -0.001376
Xavier     -0.000991
Yvonne      0.002613
Zelda       0.000713
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.26 s, sys: 87.9 ms, total: 1.35 s
Wall time: 1.08 s


name
Alice      -0.000854
Bob         0.002710
Charlie     0.000194
Dan        -0.001115
Edith       0.000555
Frank      -0.002418
George      0.001846
Hannah     -0.001776
Ingrid     -0.002040
Jerry       0.002220
Kevin      -0.000385
Laura      -0.000259
Michael    -0.001937
Norbert     0.000206
Oliver      0.000625
Patricia   -0.000618
Quinn      -0.002781
Ray         0.002357
Sarah      -0.000837
Tim         0.001247
Ursula      0.000153
Victor      0.000058
Wendy      -0.001376
Xavier     -0.000991
Yvonne      0.002613
Zelda       0.000713
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html